Aluno: Erlon Lacerda  
Matrícula: 20220071286  

In [60]:
import polars as pl
import re
import os
from collections import Counter
import random

## Funções Auxiliares

In [61]:
def get_next_word_with_most_prob(ngram_counts, w1):
    new_ngrams = [bgc for bgc in ngram_counts if bgc[0] == w1]
    most_probable = Counter(new_ngrams).most_common(1)

    return most_probable[0][0][1]

In [62]:
def get_next_word_with_random_choice_top10(ngram_counts, w1):
    new_ngrams = [bgc for bgc in ngram_counts if bgc[0] == w1]
    most_probables = Counter(new_ngrams).most_common(10)
    
    choice = random.choice(list(most_probables))[0][1]

    return choice

In [63]:
def get_next_word_with_least_prob(ngram_counts, w1):
    new_ngrams = [bgc for bgc in ngram_counts if bgc[0] == w1]
    least_prob = Counter(new_ngrams).most_common()[-1]

    return least_prob[0][1]

In [64]:
def get_next_word_randomly(ngram_counts, w1):
    new_ngrams = [bgc for bgc in ngram_counts if bgc[0] == w1]
    random_choice = random.choice(new_ngrams)

    return random_choice[1]

In [67]:
def get_ngram_counts(words, n):
    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    ngram_counts = Counter(ngrams)

    return ngram_counts

In [ ]:
def extract_words(df_pt):
    all_text = " ".join(df_pt['paragraph'].to_list())
    words = re.findall(r'\b\w+\b|[!?\.,;#\-\'\"]+', all_text.lower())
    return words

## Classe

In [74]:
class TextGenerator:
    def __init__(self, words, n=2):
        """
        Inicializa o gerador de texto.
        
        Args:
            words: Lista de palavras extraídas
            n: Tamanho do n-grama (padrão: 2 para bigramas)
        """
        self.words = words
        self.n = n
        self.ngram_counts = self.get_ngram_counts()
        
        self.CAP_PONCTS = [".", "!", "?"]
        self.NORMAL_PONCTS = [",", "\'", '\"', ";"]
        self.DASH_PONCT = "-"
    
    def get_ngram_counts(self):
        """Gera contagem de n-gramas"""
        ngrams = [tuple(self.words[i:i+self.n]) for i in range(len(self.words)-self.n+1)]
        return Counter(ngrams)
    
    def join_words(self, word_list):
        """Junta palavras aplicando regras de pontuação e capitalização"""
        word_set = {i: word for i, word in enumerate(word_list)}
        result_string = ""
        
        for i, word in list(word_set.items()):
            if i == 0:
                result_string += f"{word.capitalize()}"
                continue
            
            if word in self.CAP_PONCTS:
                result_string += f"{word}"
                if i + 1 in word_set:
                    word_set[i+1] = word_set[i+1].capitalize()
            elif word in self.NORMAL_PONCTS:
                result_string += f"{word}"
            else:
                result_string += f" {word}"
        
        return result_string
    
    def generate_with_stop(self, start_word, stop_word, choice_func, max_words=100):
        """Gera texto até encontrar palavra de parada"""
        next_word = start_word
        word_index = 0
        phrase_as_list = []
        
        while next_word != stop_word:
            if word_index >= max_words:
                break
            phrase_as_list.append(next_word)
            next_word = choice_func(self.ngram_counts, next_word)
            word_index += 1
        
        phrase_as_list.append(next_word)
        return self.join_words(phrase_as_list)
    
    def generate_n_words(self, start_word, n, choice_func):
        """Gera n palavras começando de start_word"""
        next_word = start_word
        word_index = 0
        phrase_as_list = []
        
        while word_index < n:
            phrase_as_list.append(next_word)
            next_word = choice_func(self.ngram_counts, next_word)
            word_index += 1
        
        phrase_as_list.append(next_word)
        return self.join_words(phrase_as_list)

## Uso

In [103]:
current_dir = os.getcwd()

ptbr_path = os.path.join(current_dir, "roust_dataset_ptbr.csv")
eng_path = os.path.join(current_dir, "poust_dataset_ENG.csv")

ptbr_path_exists = os.path.exists(ptbr_path)
eng_path_exists = os.path.exists(eng_path)

if not ptbr_path_exists or not eng_path_exists:
    ptbr_path = "https://raw.githubusercontent.com/erlonL/pln-2025-2/refs/heads/main/19-11-25-atividade-modelo/proust_dataset_ptbr.csv"
    eng_path = "https://raw.githubusercontent.com/erlonL/pln-2025-2/refs/heads/main/19-11-25-atividade-modelo/proust_dataset_ENG.csv"

In [104]:
df_pt = pl.read_csv(ptbr_path, separator="@")
display(df_pt.shape)
df_pt.head()

(44389, 4)

,paragraph,volume,chapter
i64,str,i64,i64
0,"""Durante muito tempo, costumava…",1,1
1,"""Apoiava brandamente minhas fac…",1,1
2,"""Tornava a adormecer, e às veze…",1,1
3,"""Às vezes, como nasceu Eva de u…",1,1
4,"""Um homem que dorme mantém em c…",1,1


In [105]:
df_en = pl.read_csv(eng_path, separator="@")
display(df_en.shape)
df_en.head()

(4416, 4)

,paragraph,volume,chapter
i64,str,i64,i64
0,"""For a long time I used to go t…",1,1
1,"""I would ask myself what o’cloc…",1,1
2,"""I would lay my cheeks gently a…",1,1
3,"""I would fall asleep, and often…",1,1
4,"""Sometimes, too, just as Eve wa…",1,1


## n = 2

In [ ]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=2)

print(tg.generate_n_words("-", 20, get_next_word_with_most_prob))
print(tg.generate_n_words("-", 20, get_next_word_with_least_prob))
print(tg.generate_n_words("-", 20, get_next_word_randomly))
print(tg.generate_n_words("-", 20, get_next_word_with_random_choice_top10))

- me cedo. às vezes, costumava deitar - me cedo. às vezes, costumava deitar - me cedo
- apagam a desenrolar dos mascarados à certas harmonias, épocas tão restrito, épocas tão restrito, épocas tão restrito
- arriscou a freqüentes e vigilante atenção à escada principal atrativo particular pense bem antes bloch e soltavam uma vitrine;
- os móveis renascença, mas o viajante o que a excitação do regresso, costumava ler às mais disparatadas,


In [ ]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=2)

print(tg.generate_with_stop("eu", ".", get_next_word_with_most_prob, max_words=20))
print(tg.generate_with_stop("eu", ".", get_next_word_with_least_prob, max_words=20))
print(tg.generate_with_stop("eu", ".", get_next_word_randomly, max_words=20))
print(tg.generate_with_stop("eu", ".", get_next_word_with_random_choice_top10, max_words=20))

Eu nem tinha tempo, costumava deitar - me cedo.
Eu interrompesse a desenrolar dos mascarados à certas harmonias, épocas tão restrito, épocas tão restrito, épocas tão restrito
Eu enganava, tonificantes à assistência olhares ambíguos, século vi os tem ainda arrastavam não disse respondeu certa me respondia
Eu adormecera; os passos de procurar dormir e as distâncias quilométricas, a vela.


In [ ]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=2)

print(tg.generate_with_stop("costumava", ".", get_next_word_with_most_prob, max_words=20))
print(tg.generate_with_stop("costumava", ".", get_next_word_with_least_prob, max_words=20))
print(tg.generate_with_stop("costumava", ".", get_next_word_randomly, max_words=20))
print(tg.generate_with_stop("costumava", ".", get_next_word_with_random_choice_top10, max_words=20))

Costumava deitar - me cedo.
Costumava esconder nas encruzilhadas, épocas tão restrito, épocas tão restrito, épocas tão restrito, épocas tão restrito,
Costumava ocultar seus fiéis na ternura apenas fora ouvido; confio o adivinhara justamente talento na luz antes disposta como bolhas
Costumava esfregar das portas misteriosamente para abrir e as belas, meus filhos de uma parte nela e carlos, mal


In [107]:
words = extract_words(df_en)
tg = TextGenerator(words, n=2)

inicial = random.choice(words)
stop = random.choice(words)

print(tg.generate_with_stop(inicial, stop, get_next_word_with_most_prob, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_least_prob, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_randomly, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_random_choice_top10, 20))

The thought that i used to go to go to go to go to go to go to go to go
The hardly approachable summit and prolongs in obsessions and prolongs in obsessions and prolongs in obsessions and prolongs in obsessions and
The anemones of orchids, watching themselves conspicuously visible by rising impulsively from carriages come under widely from sailors from fine
The fact, i was no great cloak or from heaven; i could see the book instead for ever heard


## n = 3

In [ ]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=3)

print(tg.generate_n_words(random.choice(words), 20, get_next_word_with_most_prob))
print(tg.generate_n_words(random.choice(words), 20, get_next_word_with_least_prob))
print(tg.generate_n_words(random.choice(words), 20, get_next_word_randomly))
print(tg.generate_n_words(random.choice(words), 20, get_next_word_with_random_choice_top10))

A vela, costumava deitar - me cedo. às vezes, costumava deitar - me cedo. às vezes,
Que atingem simultaneamente, épocas tão distantes vividas por lhes é reservado no seu cimo vertiginoso e julgava não deixaria eu
Os de nós, quais pode ao chegarmos à janela, sem a introduzir nenhum inconveniente e amontoavam as raias da
Atrás de encontrar algum freguês, costumava seguidamente, mas sua escada a rivalidade se apressa porque ela, não chocava


In [ ]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=3)

inicial = random.choice(words)

print(tg.generate_n_words(inicial, 20, get_next_word_with_most_prob))
print(tg.generate_n_words(inicial, 20, get_next_word_with_least_prob))
print(tg.generate_n_words(inicial, 20, get_next_word_randomly))
print(tg.generate_n_words(inicial, 20, get_next_word_with_random_choice_top10))

Distância, costumava deitar - me cedo. às vezes, costumava deitar - me cedo. às vezes, costumava
Distância que atingem simultaneamente, épocas tão distantes vividas por lhes é reservado no seu cimo vertiginoso e julgava não deixaria
Distância quilométrica de x. quero referir - apagam - nosso espírito de homem que se feliz. resignavam com uma
Distância da obra destacava em uma parte. e repousante doçura próxima vez de pensar nisso de pensar nisso que já


In [ ]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=3)

inicial = random.choice(words)
stop = random.choice(words)

print(tg.generate_with_stop(inicial, stop, get_next_word_with_most_prob, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_least_prob, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_randomly, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_random_choice_top10, 20))

Ao despertar; queria largar o volume que eu nem tinha tempo,
Ao sentir que atingem simultaneamente,
Ao pescoço firme,
Ao adormecer,


In [108]:
words = extract_words(df_en)
tg = TextGenerator(words, n=3)

inicial = random.choice(words)
stop = random.choice(words)

print(tg.generate_with_stop(inicial, stop, get_next_word_with_most_prob, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_least_prob, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_randomly, 20))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_random_choice_top10, 20))

Myself seemed actually to go to go to go to go to go to go to go to go to go
Myself. for long a leaf on growing, simultaneously touching widely separated years and the distant periods they stand like
Myself of gaiety remains external. de marsantes had never to practise it about midday, loyalty, whether this had
Myself in my book would form upon the candle the rivalry. oh, was effectually dispelled, and half -


## n = 20

In [110]:
words = extract_words(df_pt)
tg = TextGenerator(words, n=20)

inicial = random.choice(words)

print(tg.generate_n_words(inicial, 50, get_next_word_with_most_prob))
print(tg.generate_n_words(inicial, 50, get_next_word_with_least_prob))
print(tg.generate_n_words(inicial, 50, get_next_word_randomly))
print(tg.generate_n_words(inicial, 50, get_next_word_with_random_choice_top10))

Pareceria um tanto particular; queria largar o volume que eu nem tinha tempo, costumava deitar - me cedo. às vezes, costumava deitar - me cedo. às vezes, costumava deitar - me cedo. às vezes, costumava deitar - me cedo. às vezes,
Pareceria impossível de nela descrever os faria se ocupassem um lugar, épocas tão restrito que atingem simultaneamente, épocas tão restrito que atingem simultaneamente, épocas tão restrito que atingem simultaneamente, épocas tão restrito que atingem simultaneamente, épocas tão restrito que atingem simultaneamente, épocas tão restrito que
Pareceria uma mulher que tal como françoís le champi! a minha sobrinha. na mão, em tributo a ser muito feliz, toda a vista da rua transversal entre os progressos fossem convidadas, a verdade que eu já flutuar perpetuamente uma visita e alumiadas pelo progresso nas duas
Pareceria um quarteto. 5 essa escadaria dos débates roses, meia. é - se de pensar diferente daquele sábado, um pássaro a ele do local de ler a vela com aquela p

In [109]:
words = extract_words(df_en)
tg = TextGenerator(words, n=20)

inicial = random.choice(words)
stop = random.choice(words)

print(tg.generate_with_stop(inicial, stop, get_next_word_with_most_prob, 50))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_least_prob, 50))
print(tg.generate_with_stop(inicial, stop, get_next_word_randomly, 50))
print(tg.generate_with_stop(inicial, stop, get_next_word_with_random_choice_top10, 50))

Become the thought that i used to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to go to
Become importunate. if need be, simultaneously touching widely separated years and the distant periods they have lived through between which so many days and the distant periods they have lived through between which so many days and the distant periods they have lived through between which so many days
Become cultured, a party, and such - starred here, for you, was now more wednesdays, which in my bedroom floors above him upon those that special sauce, this same person whom we had declined to albertine imparted sincerity towards her eyes blazing houses of suspecting
Become one of it appeared embarrassed at my book instead, a state, was asleep, of their nests, but my hands or suppose that old days, a channel say to bed, of their folds. and even completely dispelled, but will never failed to have
